In [3]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# 1) 读取数据
data = pd.read_parquet("database.parquet") #加载原始数据
data = data.rename(columns={'assigned_domain': 'assigned_domain_by_embedding'})

# 2) 每个类别随机抽取最多 1000 条
sampled = (
    data.groupby("assigned_domain_by_embedding", group_keys=False)
        .apply(lambda g: g.sample(n=min(len(g), 2000), random_state=42))
        .reset_index(drop=True)
)
# sampled['assigned_domain_by_embedding'].value_counts()

# 3) OpenAI 客户端
client = OpenAI(
    base_url="https://api.kwwai.top/v1",
    api_key="sk-ba5wabFMCaXe5O4LyhIxyh9JtWQXbuMDhGEWDHrRwRMCCbMt"
)

categories = [
    "mathematical and physical science",
    "chemical science",
    "life science",
    "engineering and materials science",
    "earth science",
    "information science",
    "management science",
    "health science"
]

# 4) 定义分类函数（带断点保存）
def classify_and_save(df, model_name, col_name, batch_size=10, out_file="LLM分类结果.parquet"):
    if col_name not in df:
        df[col_name] = None

    for i, text in enumerate(tqdm(df["abstract"], desc=f"Classifying with {model_name}")):
        # 跳过已算过的
        if pd.notna(df.loc[i, col_name]):
            print(f"""对于{col_name}, 第{i+1}条数据已经存在""")
            continue
        
        prompt = f"""
        请将下面的学术论文摘要归类到以下8个学科领域中的一个：
        {categories}

        文本内容：
        {text}

        只输出对应的学科名称，不要输出其他解释。
        """
        try:
            resp = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}]
            )
            df.loc[i, col_name] = resp.choices[0].message.content.strip()
        except Exception as e:
            # 出错时打印这一行的信息
            print(f"\n❌ 第 {i+1} 行出错: {e}")
            print(f"摘要内容: {text}\n")
            df.loc[i, col_name] = None  # 标记为 None，继续下一行
            
        # 每 batch_size 保存一次
        if (i + 1) % batch_size == 0:
            df.to_parquet(out_file, index=False)

    # 最后再保存一次
    df.to_parquet(out_file, index=False)
    return df

df = pd.read_parquet("LLM分类结果.parquet")

# 5) 分别用 gpt-4o 和 gpt-5 分类并保存
df= classify_and_save(df, "gpt-4o", "domain_by_gpt4o")
df = classify_and_save(df, "gpt-5", "domain_by_gpt5")
df = classify_and_save(df, "DeepSeek-R1", "domain_by_DeepSeek-R1")

C:\Users\lipen\AppData\Local\Temp\ipykernel_72812\3603624333.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min(len(g), 2000), random_state=42))
Classifying with gpt-4o: 100%|██████████| 16000/16000 [00:00<00:00, 126064.15it/s]


对于domain_by_gpt4o, 第1条数据已经存在
对于domain_by_gpt4o, 第2条数据已经存在
对于domain_by_gpt4o, 第3条数据已经存在
对于domain_by_gpt4o, 第4条数据已经存在
对于domain_by_gpt4o, 第5条数据已经存在
对于domain_by_gpt4o, 第6条数据已经存在
对于domain_by_gpt4o, 第7条数据已经存在
对于domain_by_gpt4o, 第8条数据已经存在
对于domain_by_gpt4o, 第9条数据已经存在
对于domain_by_gpt4o, 第10条数据已经存在
对于domain_by_gpt4o, 第11条数据已经存在
对于domain_by_gpt4o, 第12条数据已经存在
对于domain_by_gpt4o, 第13条数据已经存在
对于domain_by_gpt4o, 第14条数据已经存在
对于domain_by_gpt4o, 第15条数据已经存在
对于domain_by_gpt4o, 第16条数据已经存在
对于domain_by_gpt4o, 第17条数据已经存在
对于domain_by_gpt4o, 第18条数据已经存在
对于domain_by_gpt4o, 第19条数据已经存在
对于domain_by_gpt4o, 第20条数据已经存在
对于domain_by_gpt4o, 第21条数据已经存在
对于domain_by_gpt4o, 第22条数据已经存在
对于domain_by_gpt4o, 第23条数据已经存在
对于domain_by_gpt4o, 第24条数据已经存在
对于domain_by_gpt4o, 第25条数据已经存在
对于domain_by_gpt4o, 第26条数据已经存在
对于domain_by_gpt4o, 第27条数据已经存在
对于domain_by_gpt4o, 第28条数据已经存在
对于domain_by_gpt4o, 第29条数据已经存在
对于domain_by_gpt4o, 第30条数据已经存在
对于domain_by_gpt4o, 第31条数据已经存在
对于domain_by_gpt4o, 第32条数据已经存在
对于domain_by_gpt4o, 第33条数据已经存在
对于domain_by_gpt4o, 

Classifying with gpt-5:  77%|███████▋  | 12328/16000 [00:00<00:00, 122371.01it/s]

对于domain_by_gpt5, 第1条数据已经存在
对于domain_by_gpt5, 第2条数据已经存在
对于domain_by_gpt5, 第3条数据已经存在
对于domain_by_gpt5, 第4条数据已经存在
对于domain_by_gpt5, 第5条数据已经存在
对于domain_by_gpt5, 第6条数据已经存在
对于domain_by_gpt5, 第7条数据已经存在
对于domain_by_gpt5, 第8条数据已经存在
对于domain_by_gpt5, 第9条数据已经存在
对于domain_by_gpt5, 第10条数据已经存在
对于domain_by_gpt5, 第11条数据已经存在
对于domain_by_gpt5, 第12条数据已经存在
对于domain_by_gpt5, 第13条数据已经存在
对于domain_by_gpt5, 第14条数据已经存在
对于domain_by_gpt5, 第15条数据已经存在
对于domain_by_gpt5, 第16条数据已经存在
对于domain_by_gpt5, 第17条数据已经存在
对于domain_by_gpt5, 第18条数据已经存在
对于domain_by_gpt5, 第19条数据已经存在
对于domain_by_gpt5, 第20条数据已经存在
对于domain_by_gpt5, 第21条数据已经存在
对于domain_by_gpt5, 第22条数据已经存在
对于domain_by_gpt5, 第23条数据已经存在
对于domain_by_gpt5, 第24条数据已经存在
对于domain_by_gpt5, 第25条数据已经存在
对于domain_by_gpt5, 第26条数据已经存在
对于domain_by_gpt5, 第27条数据已经存在
对于domain_by_gpt5, 第28条数据已经存在
对于domain_by_gpt5, 第29条数据已经存在
对于domain_by_gpt5, 第30条数据已经存在
对于domain_by_gpt5, 第31条数据已经存在
对于domain_by_gpt5, 第32条数据已经存在
对于domain_by_gpt5, 第33条数据已经存在
对于domain_by_gpt5, 第34条数据已经存在
对于domain_by_gpt5, 第35条数

Classifying with gpt-5: 100%|██████████| 16000/16000 [00:00<00:00, 121134.04it/s]


对于domain_by_gpt5, 第12329条数据已经存在
对于domain_by_gpt5, 第12330条数据已经存在
对于domain_by_gpt5, 第12331条数据已经存在
对于domain_by_gpt5, 第12332条数据已经存在
对于domain_by_gpt5, 第12333条数据已经存在
对于domain_by_gpt5, 第12334条数据已经存在
对于domain_by_gpt5, 第12335条数据已经存在
对于domain_by_gpt5, 第12336条数据已经存在
对于domain_by_gpt5, 第12337条数据已经存在
对于domain_by_gpt5, 第12338条数据已经存在
对于domain_by_gpt5, 第12339条数据已经存在
对于domain_by_gpt5, 第12340条数据已经存在
对于domain_by_gpt5, 第12341条数据已经存在
对于domain_by_gpt5, 第12342条数据已经存在
对于domain_by_gpt5, 第12343条数据已经存在
对于domain_by_gpt5, 第12344条数据已经存在
对于domain_by_gpt5, 第12345条数据已经存在
对于domain_by_gpt5, 第12346条数据已经存在
对于domain_by_gpt5, 第12347条数据已经存在
对于domain_by_gpt5, 第12348条数据已经存在
对于domain_by_gpt5, 第12349条数据已经存在
对于domain_by_gpt5, 第12350条数据已经存在
对于domain_by_gpt5, 第12351条数据已经存在
对于domain_by_gpt5, 第12352条数据已经存在
对于domain_by_gpt5, 第12353条数据已经存在
对于domain_by_gpt5, 第12354条数据已经存在
对于domain_by_gpt5, 第12355条数据已经存在
对于domain_by_gpt5, 第12356条数据已经存在
对于domain_by_gpt5, 第12357条数据已经存在
对于domain_by_gpt5, 第12358条数据已经存在
对于domain_by_gpt5, 第12359条数据已经存在
对于domain

Classifying with DeepSeek-R1:  85%|████████▌ | 13621/16000 [00:00<00:00, 135456.36it/s]

对于domain_by_DeepSeek-R1, 第1条数据已经存在
对于domain_by_DeepSeek-R1, 第2条数据已经存在
对于domain_by_DeepSeek-R1, 第3条数据已经存在
对于domain_by_DeepSeek-R1, 第4条数据已经存在
对于domain_by_DeepSeek-R1, 第5条数据已经存在
对于domain_by_DeepSeek-R1, 第6条数据已经存在
对于domain_by_DeepSeek-R1, 第7条数据已经存在
对于domain_by_DeepSeek-R1, 第8条数据已经存在
对于domain_by_DeepSeek-R1, 第9条数据已经存在
对于domain_by_DeepSeek-R1, 第10条数据已经存在
对于domain_by_DeepSeek-R1, 第11条数据已经存在
对于domain_by_DeepSeek-R1, 第12条数据已经存在
对于domain_by_DeepSeek-R1, 第13条数据已经存在
对于domain_by_DeepSeek-R1, 第14条数据已经存在
对于domain_by_DeepSeek-R1, 第15条数据已经存在
对于domain_by_DeepSeek-R1, 第16条数据已经存在
对于domain_by_DeepSeek-R1, 第17条数据已经存在
对于domain_by_DeepSeek-R1, 第18条数据已经存在
对于domain_by_DeepSeek-R1, 第19条数据已经存在
对于domain_by_DeepSeek-R1, 第20条数据已经存在
对于domain_by_DeepSeek-R1, 第21条数据已经存在
对于domain_by_DeepSeek-R1, 第22条数据已经存在
对于domain_by_DeepSeek-R1, 第23条数据已经存在
对于domain_by_DeepSeek-R1, 第24条数据已经存在
对于domain_by_DeepSeek-R1, 第25条数据已经存在
对于domain_by_DeepSeek-R1, 第26条数据已经存在
对于domain_by_DeepSeek-R1, 第27条数据已经存在
对于domain_by_DeepSeek-R1, 第28条数据已经存在
对

Classifying with DeepSeek-R1: 100%|██████████| 16000/16000 [00:00<00:00, 129307.91it/s]


In [4]:
1

1